In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [72]:
import torch
import sklearn

In [73]:
# seed 고정
torch.manual_seed(1)

In [74]:
# data load
train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/submit_sample.csv")

In [75]:
# data 확인
print(train.head())
print(test.head())
print(submit.head())

In [76]:
# 데이터 전처리 및 텐서 올리기
# 필요없는 컬럼 삭제
x_train = np.array(train.drop(['0','0.1'],axis=1))
x_test =np.array(test.drop(['0'],axis=1))
y_train = np.array(train['0.1'])


# 분류 category를 위한 인코딩(라벨 인코딩)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()
y_train = torch.LongTensor(y_train).cuda()

In [77]:
# data 모양 확인
print(x_train.shape,y_train.shape)

In [83]:
# 학습 설정
epochs = 100000
w = torch.zeros((150,len(le.classes_)),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")
optim = torch.optim.SGD([w,b],lr=2e-5)
# 여러개의 클라스 분류 문제 이므로 cross Entropy 사용
loss = torch.nn.CrossEntropyLoss()

In [84]:
# 학습
for epoch in range(epochs + 1):
    optim.zero_grad()
    
    h = x_train.matmul(w)+b
    cost = loss(h,y_train)
    
    cost.backward()
    optim.step()
    if epoch % 10000 == 0:
        print(cost.item())

In [80]:
with torch.no_grad():
    predict = torch.softmax(x_test.matmul(w)+b,dim=1)
    predict = torch.argmax(predict,dim=1)
predict = le.inverse_transform(predict.cpu())
submit['Category'] = predict
print(submit)
submit.to_csv("submission.csv",index=False)

# try 
# y_train의 차원 설정을 1로해서 cross entropy에서 오류 발생
# lr = 1e-4 는 cost가 진동함 
# lr = 1e-3 도 진동함 -> 러닝 레이트가 너무 크다!
# lr = 1e-5 에서 1.7 정도로 고정되는 것이 확인 된다. -> 리더보드에서 정확도가 제대로 나오지 않음
# lr = 2e-5 And Epoch = 100000 -> 에포크 증가 및 러닝레이트 살짝 증가